# Train Translator

Based on: https://huggingface.co/docs/transformers/tasks/translation

In [1]:
import sys, os, datetime
import json
import torch
import random
import glob
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import TranslationPipeline
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

2023-03-19 10:12:54.194753: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 10:12:54.308288: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-19 10:12:54.706047: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:
2023-03-19 10:12:54.706091: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
import cdli
import languages

In [3]:
def get_finetune_model_id(model_id):
    model_dir = f"../results/{model_id}"
    checkpoints = [(os.path.abspath(x), int(os.path.split(x)[1].split("-")[1])) for x in glob.glob(f"{model_dir}/checkpoint-*")]
    checkpoints = sorted(checkpoints, key=lambda x: x[1])[-1]
    return checkpoints[0]

In [4]:
os.environ["WANDB_NOTEBOOK_NAME"] = "TrainTranslatorNew.ipynb"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

source_langs = set(["akk", "sux"])

# target_langs = set(["en", "it", "es", "fr", "de"])
target_langs = set(["en"])

base_model_id = "t5-base"
finetune_model_id = None
# finetune_model_id = get_finetune_model_id("t5-base-p-akksux-en-20220722-173018")

model_max_length = 512
batch_size = 4 if os.path.basename(base_model_id).startswith("t5-large") else (8 if os.path.basename(base_model_id).startswith("t5-base") else 128)

num_train_epochs = 30

is_bi = True
is_finetune = finetune_model_id is not None and len(finetune_model_id) > 1

In [5]:
date_id = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
flags = ""
suffix = ""
if is_bi:
    flags += "-bi"
if is_finetune:
    flags += "-f"
    suffix += f"-{os.path.basename(os.path.split(finetune_model_id)[0])}-{os.path.basename(finetune_model_id)}"
model_id = f"{os.path.basename(base_model_id)}{flags}-{''.join(sorted(list(source_langs)))}-{''.join(sorted(list(target_langs)))}-{date_id}{suffix}"
model_id

't5-base-bi-akksux-en-20230319-101322'

In [6]:
has_cuda = torch.cuda.is_available()
device = torch.cuda.device(0) if has_cuda else "cpu"
has_cuda, device

(True, <torch.cuda.device at 0x7fd32c521900>)

In [7]:
!nvidia-smi

Sun Mar 19 10:13:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   43C    P8    35W / 350W |    151MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Training Data

In [8]:
def get_prefix(src_lang, tgt_lang):
    s = languages.all_languages[src_lang]
    t = languages.all_languages[tgt_lang]
    return f"translate {s} to {t}: "
    
get_prefix("suxts", "es")

'translate Sumerian to Spanish: '

In [9]:
def load_translations_dataset(from_lang, to_lang):
    ds = Dataset.from_json(f"../data/translations_{from_lang}_to_{to_lang}.jsonl")
    srcs = [get_prefix(from_lang, to_lang) + x[from_lang] for x in ds]
    targets = [x[to_lang] for x in ds]
    if is_bi:
        srcs.extend([get_prefix(to_lang, from_lang) + x[to_lang] for x in ds])
        targets.extend([x[from_lang] for x in ds])
#     ds = ds.add_column("source", srcs)
#     ds = ds.add_column("target", targets)
#     ds = ds.remove_columns([from_lang])
#     ds = ds.rename_column(to_lang, "target")
    ds = Dataset.from_dict({"source": srcs, "target": targets})
    return ds
    
translation_datasets = {lang: load_translations_dataset(lang, "en") for lang in source_langs}

Using custom data configuration default-5d510aca781cdbde
Reusing dataset json (/home/fak/.cache/huggingface/datasets/json/default-5d510aca781cdbde/0.0.0)
Using custom data configuration default-f1f92e2230afbcae
Reusing dataset json (/home/fak/.cache/huggingface/datasets/json/default-f1f92e2230afbcae/0.0.0)


In [10]:
translation_datasets

{'sux': Dataset({
     features: ['source', 'target'],
     num_rows: 118838
 }),
 'akk': Dataset({
     features: ['source', 'target'],
     num_rows: 231802
 })}

In [14]:
translation_datasets["sux"][1]

{'source': 'translate Sumerian to English: # = %a ru-ub-bu-u2 # e3 ~ = %a e-ru-u2-um # e3 ~ = %a a-ma-<<x>>-rum',
 'target': 'to bring up ? to watch'}

In [15]:
merged_dataset = concatenate_datasets(translation_datasets.values())

In [16]:
merged_dataset

Dataset({
    features: ['source', 'target'],
    num_rows: 350640
})

In [17]:
dataset = merged_dataset.shuffle()

In [18]:
dataset[1]

{'source': 'translate English to Sumerian: radish seed',
 'target': '[numun] puglu(_sza3-gi_)#{sar}'}

In [19]:
avg_src_chars_per_token = 1.8338974021110785
avg_tgt_chars_per_token = 2.829482016086902

In [20]:
def target_ok(target_text):
    if len(target_text) == 0:
        return False
    if len(set(target_text.replace(" ", ""))) < 2:
        return False
    return True
    

def test_target_ok(text):
    ok = target_ok(text)
    print(ok, repr(text))
    
test_target_ok("")
test_target_ok(" ")
test_target_ok("xx xxx x")
test_target_ok(".. . .. ")
test_target_ok("Hi")

False ''
False ' '
False 'xx xxx x'
False '.. . .. '
True 'Hi'


In [21]:
translations = dataset.train_test_split(test_size=0.1)

In [22]:
train_dataset, test_dataset = translations["train"], translations["test"]

In [23]:
test_dataset

Dataset({
    features: ['source', 'target'],
    num_rows: 35064
})

In [24]:
test_dataset[1120:1200]

{'source': ["translate Akkadian to English: [...] _sza3_-szu2 _sig2 babbar e3_{a} _nu_ x [...] [... bu-bu-u']-ta _sa5_ u il-la-tu-szu2 il-la#-ka bu-u'-sza2-nu _mu_-_ni#_",
  'translate English to Akkadian: Thirty-two statues of kings fashioned from silver, gold, copper, and alabaster from inside the cities Susa, Madaktu, and Ḫurādi, along with a statue of Ummanigaš Ḫumban-nikaš I, son of Umbadarâ, a statue of Ištar-Nanḫūndi Šutur-Naḫūndi II, a statue of Ḫallušu Ḫallušu-Inšušinak I, and a statue of the later Tammarītu, who did obeisance to me by the command of the god Aššur and the goddess Ištar — I took them to Assyria.',
  'translate Akkadian to English: qab2-li a-ga-de3{ki} _e2_-su ib-ni-u3 sza _dub_ su4-a u-sa-sa3-ku-ni {d}utu u3 {d}inanna u3 {d}ne3-iri11-gal _maszkim_ _lugal_ u3',
  'translate English to Sumerian: I want to cool the inhabitant of the Etemenanki temple',
  'translate English to Akkadian: In the Path of those i.e. stars of Ea and the ones who . . . . . .',
  'transla

## Tokenize the Data

In [25]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, model_max_length=model_max_length)

In [26]:
tokenizer.model_max_length

512

In [27]:
print("pad", tokenizer.pad_token, tokenizer.pad_token_id)
print("eos", tokenizer.eos_token, tokenizer.eos_token_id)
print("unk", tokenizer.unk_token, tokenizer.unk_token_id)

pad <pad> 0
eos </s> 1
unk <unk> 2


In [28]:
ccc = 0
sum_src_chars_per_token = 0.0
num_src_chars_per_token = 0
sum_tgt_chars_per_token = 0.0
num_tgt_chars_per_token = 0

def preprocess_function(examples):
    global ccc, sum_src_chars_per_token, sum_tgt_chars_per_token, num_src_chars_per_token, num_tgt_chars_per_token
#     print(examples)
    inputs = [example for example in examples["source"]]
    targets = [example for example in examples["target"]]
    model_inputs = tokenizer(inputs, max_length=model_max_length, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=model_max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    nexamples = len(inputs)
    for i in range(nexamples):
        nchar = len(inputs[i])
        ntoks = len(model_inputs["input_ids"][i])
        if ntoks > 0:
            sum_src_chars_per_token += nchar / ntoks
            num_src_chars_per_token += 1
        nchar = len(targets[i])
        ntoks = len(model_inputs["labels"][i])
        if ntoks > 0:
            sum_tgt_chars_per_token += nchar / ntoks
            num_tgt_chars_per_token += 1
    
    ccc += 1
    if ccc == 1:
        print(model_inputs["input_ids"][0])
        print(model_inputs["labels"][0])
        nchar = len(targets[0])
        ntoks = len(model_inputs["labels"][0])
        print(nchar, ntoks, nchar / ntoks)

    return model_inputs

tokenized_translations = translations.map(preprocess_function, batched=True)
tokenized_translations

Parameter 'function'=<function preprocess_function at 0x7fd28e250820> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/316 [00:00<?, ?ba/s]

[13959, 12198, 15, 5288, 12, 1566, 10, 108, 102, 9, 3, 450, 18, 2, 26, 2, 13363, 51, 9, 18, 1050, 591, 3, 15, 7412, 6191, 18, 9, 18, 29, 23, 3, 2897, 172, 256, 18, 51, 23, 18, 77, 18, 2408, 18, 397, 23296, 7466, 3, 23, 52, 23, 2596, 18, 6191, 58, 18, 521, 58, 18, 1050, 591, 18, 15, 18, 29, 15, 3, 2, 26, 2, 9, 18, 29, 202, 18, 29, 9, 3, 29, 77, 26, 9, 3, 7, 1000, 7860, 18, 122, 9, 18, 115, 9, 1]
[16, 112, 20212, 8, 28743, 4575, 18, 567, 9, 635, 9, 1860, 135, 5, 37, 248, 3, 233, 13, 8, 365, 7276, 6, 8, 389, 202, 9, 6, 227, 8, 11571, 130, 2569, 6, 1]
129 35 3.6857142857142855


  0%|          | 0/36 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 315576
    })
    test: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 35064
    })
})

In [29]:
avg_src_chars_per_token = sum_src_chars_per_token / num_src_chars_per_token
avg_tgt_chars_per_token = sum_tgt_chars_per_token / num_tgt_chars_per_token
print("avg_src_chars_per_token", "=", avg_src_chars_per_token)
print("avg_tgt_chars_per_token", "=", avg_tgt_chars_per_token)

avg_src_chars_per_token = 2.584958974100154
avg_tgt_chars_per_token = 2.057410167608085


In [30]:
tokenized_translations["train"] = tokenized_translations["train"].remove_columns(["source", "target"])
tokenized_translations["test"] = tokenized_translations["test"].remove_columns(["source", "target"])
tokenized_translations

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 315576
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 35064
    })
})

In [31]:
source_max_length = max([len(x["input_ids"]) for x in tokenized_translations["train"]])
target_max_length = max([len(x["labels"]) for x in tokenized_translations["train"]])
source_max_length, target_max_length

(512, 512)

In [32]:
tokenized_translations["train"][0]["labels"][:10]

[16, 112, 20212, 8, 28743, 4575, 18, 567, 9, 635]

## Load the Model

In [33]:
model = AutoModelForSeq2SeqLM.from_pretrained(finetune_model_id if is_finetune else base_model_id, 
                                              max_length=model_max_length)

In [34]:
model.config

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 512,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to Ge

## Train

In [37]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
# data_collator

In [38]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"../results/{model_id}",
    evaluation_strategy="epoch",
    learning_rate=2.0*2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    fp16=has_cuda,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_translations["train"],
    eval_dataset=tokenized_translations["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)



Using amp half precision backend


In [39]:
trainer.train()

/home/fak/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 315576
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1183410
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: praeclarum. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.588500,1.418067
2,1.413500,1.316420
3,0.000000,nan


Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-500
Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-1000
Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-1000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-202

Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-7000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-7000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-7000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-5500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-7500
Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-7500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-7500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-7500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-7500/special_tokens_map.json
Deleting ol

Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-13500
Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-13500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-13500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-13500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-13500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-12000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-14000
Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-14000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-14000/pytorch_model.bin
tokenizer config file saved in ../results/t5-b

...

Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-101000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-101000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-101000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-101000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-99500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-101500
Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-101500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-101500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-101500/tokenizer_config.json
Special toke

Special tokens file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-107000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-105500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-107500
Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-107500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-107500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-107500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-107500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-106000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-108000
Con

Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-113500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-113500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-113500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-113500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-112000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-114000
Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-114000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-114000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-114000/tokenizer_config.json
Special tok

tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-119500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-119500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-118000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-120000
Configuration saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-120000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-120000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-120000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-120000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230318-113937/checkpoint-118500] due to

KeyboardInterrupt: 

## Sample

In [ ]:
pipeline = TranslationPipeline(model=model.to("cpu"), tokenizer=tokenizer, max_length=model_max_length)

In [ ]:
pipeline

In [ ]:
pipeline("translate English to French: hello my name is Frank")

In [ ]:
source_test = translations["test"][0]["source"]
target_test = translations["test"][0]["target"]
print(source_test)
print("-"*80)
print(target_test)

In [ ]:
def translate(text):
    return pipeline(text)

translate(source_test)

In [ ]:
tests = original_tests
def sample(num_samples=100):
    for i in range(min(num_samples, tests.num_rows)):
        t = tests[i]
    #     print(t)
        src = t["source"]
        tgt = t["target"]
        query = src
        pred = pipeline(query)[0]["translation_text"]
        print("-"*48)
        print("QUERY ", query)
        print("TARGET", tgt)
        print("PRED  ", pred)
    #     break
    
sample()

## Save to Huggingface

In [ ]:
model_path = os.path.abspath(f"/home/fak/nn/Data/generated/cuneiform/{model_id}-fullytrained")
trainer.save_model(model_path)
model_path

In [ ]:
tokenizer.save_pretrained(model_path)